In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import yfinance as yf

%matplotlib inline

In [ ]:
# Reading Temperature data and convert it to dataframe
#Data cleanup




In [ ]:
# Reading Natural gas data and convert it to dataframe
#Data cleanup
#describe()



In [ ]:
#Return close prices of Natural gas and Average temp 



In [ ]:
#join close price of natural gas and Average temprature using concatenation



In [ ]:
#Find daily return of the two joined data frame
# Drop nulls
# Count nulls



In [ ]:
#plot daily return of the close price of naturan gas and temprature
#print out any similarities and differences between them
## Box plot to visually show risk



In [ ]:
# Calculate the daily standard deviations
# Calculate the annualized standard deviation (252 trading days)



In [ ]:
# Calculate rolling standard deviation

# Plot rolling standard deviation



In [ ]:
# Calculate and plot the correlation



In [ ]:
# explation the correlation between Temp and Price of Natural gas
